In [1]:
import pandas as pd
import torch
from transformers import AlbertTokenizer, AlbertForSequenceClassification

In [2]:
file_path = '/Users/negarakhgar/Desktop/nlp project/data/english_boardgames_comments.csv'
english_comments_df = pd.read_csv(file_path)

In [3]:
aspects = { 
    'luck': ['luck', 'chance', 'random', 'alea'],
    'bookkeeping': ['recording', 'bookkeeping', 'rulebook', 'manual', 'tracking', 'record'],
    'downtime': ['downtime', 'waiting', 'idle', 'turn'],
    'interaction': ['interaction', 'influence', 'impact', 'affect'],
    'bash_the_leader': ['bash the leader', 'target the leader', 'attack the leader'],
    'complexity': ['complicated', 'complex', 'rules', 'difficult', 'complex']
}

In [4]:
model_albert = AlbertForSequenceClassification.from_pretrained('/Users/negarakhgar/Desktop/nlp project/model')
tokenizer_albert = AlbertTokenizer.from_pretrained('/Users/negarakhgar/Desktop/nlp project/model')

In [5]:
#Setting the mode to evaluation mode
model_albert.eval()

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=

In [6]:
# Aspect extraction
def extract_aspects(comment):
    detected_aspects = []
    for aspect, keywords in aspects.items():
        if any(keyword in comment.lower() for keyword in keywords):
            detected_aspects.append(aspect)
    return detected_aspects

In [7]:
# Applying aspect extraction to each comment
english_comments_df['detected_aspects'] = english_comments_df['value'].apply(extract_aspects)

In [8]:
# Filtering out comments that have detected aspects
filtered_comments_df = english_comments_df[english_comments_df['detected_aspects'].map(len) > 0].copy()

In [9]:
def absa_on_aspects(comment, detected_aspects):
    aspect_sentiments = {}
    for aspect in detected_aspects:
        inputs = tokenizer_albert(comment, padding=True, truncation=True, max_length=512, return_tensors='pt')
        inputs = {key: val for key, val in inputs.items()}

        with torch.no_grad():
            outputs = model_albert(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1).item()

        labels = {0: 'negative', 1: 'positive'}
        aspect_sentiments[aspect] = labels[predicted_class]

    return aspect_sentiments

In [10]:
# Applying ABSA to each comment with detected aspects
filtered_comments_df['aspect_sentiments'] = filtered_comments_df.apply(
    lambda row: absa_on_aspects(row['value'], row['detected_aspects']), axis=1
)

In [11]:
absa_results_file = '/Users/negarakhgar/Desktop/nlp project/boardgames_absa_results_albert.csv'
filtered_comments_df.to_csv(absa_results_file, index=False)

In [12]:
print(filtered_comments_df[['boardgame_id', 'value', 'detected_aspects', 'aspect_sentiments']].head())

print(f"ABSA completed and results saved to {absa_results_file}")

    boardgame_id                                              value  \
3         224517  Very clever game, enjoyable overall.  Plus poi...   
4         224517  Brilliant!  Fits right into my wheelhouse all ...   
7         224517  The game itself is not interesting enough to l...   
9         224517  This is a near-perfect board game because...  ...   
10        224517  Excellent and highly interactive game. Probabl...   

                       detected_aspects  \
3   [downtime, interaction, complexity]   
4                         [interaction]   
7                         [interaction]   
9             [interaction, complexity]   
10  [downtime, interaction, complexity]   

                                    aspect_sentiments  
3   {'downtime': 'positive', 'interaction': 'posit...  
4                         {'interaction': 'positive'}  
7                         {'interaction': 'negative'}  
9   {'interaction': 'positive', 'complexity': 'pos...  
10  {'downtime': 'positive', 'inte